# Adverse Weather Object Detection with YOLOv5

This notebook implements an object detection system for autonomous vehicles under adverse weather conditions (rainy, foggy, snowy) using YOLOv5s and the BDD100K dataset. It includes:

1. Filtering a subset of ~1,000 images from BDD100K with adverse weather.
2. Training YOLOv5s on this subset.
3. Evaluating the model and saving results.
4. Running inference and saving output images.
5. Saving all results for later presentation without retraining.

**Hardware**: RTX 3050 4GB VRAM, Ryzen 7 4800H, 16GB RAM, 90GB free storage.


In [1]:
# Install dependencies and clone YOLOv5 (run only if not already installed)
!pip install torch==2.3.0+cu118 torchvision==0.18.0+cu118 --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python albumentations pyyaml tqdm

Looking in indexes: https://download.pytorch.org/whl/cu118


## Step 1: Filter BDD100K Dataset for Adverse Weather

Filter ~1,000 images from the BDD100K dataset (train split) with adverse weather conditions (rainy, foggy, snowy). Save them to `dataset/images/train` with dummy labels in `dataset/labels/train`.

In [2]:
import json
import os
import shutil
from pathlib import Path
import random
import cv2

# Paths
BDD100K_ROOT = "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/bdd100k"
OUTPUT_DIR = "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/dataset"
MAX_IMAGES_TRAIN = 1000  # Number of training images
MAX_IMAGES_VAL = 200     # Number of validation images
WEATHER_CLASSES = ["rainy", "foggy", "snowy"]
YOLO_CLASSES = {"car": 0}  # Add more classes if needed

def convert_bbox_to_yolo(bbox, img_width, img_height):
    x1, y1, x2, y2 = bbox
    x_center = ((x1 + x2) / 2.0) / img_width
    y_center = ((y1 + y2) / 2.0) / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    return x_center, y_center, width, height

def filter_adverse_weather(root_dir, output_dir, max_images_train, max_images_val):
    # Process both train and val splits
    for split in ["train", "val"]:
        images_dir = os.path.join(root_dir, f"{split}/images")
        labels_file = os.path.join(root_dir, f"{split}/annotations/bdd100k_labels_images_{split}.json")
        output_images = os.path.join(output_dir, f"images/{split}")
        output_labels = os.path.join(output_dir, f"labels/{split}")

        # Create output directories
        os.makedirs(output_images, exist_ok=True)
        os.makedirs(output_labels, exist_ok=True)

        # Check if label file exists
        if not os.path.exists(labels_file):
            print(f"Label file not found for {split} split: {labels_file}")
            continue

        # Load annotations
        with open(labels_file, 'r') as f:
            data = json.load(f)

        # Filter for adverse weather
        adverse_data = [entry for entry in data if entry.get("attributes", {}).get("weather", "clear") in WEATHER_CLASSES]

        # Set max images based on split
        max_images = max_images_train if split == "train" else max_images_val
        if len(adverse_data) == 0:
            print(f"No adverse weather images found in {split} split.")
            continue

        selected = random.sample(adverse_data, min(max_images, len(adverse_data)))

        for entry in selected:
            image_name = entry["name"]
            src_img_path = os.path.join(images_dir, image_name)
            dst_img_path = os.path.join(output_images, image_name)

            if not os.path.exists(src_img_path):
                print(f"Image not found: {src_img_path}")
                continue

            # Load image to get dimensions
            img = cv2.imread(src_img_path)
            if img is None:
                print(f"Warning: Could not load image {src_img_path}, skipping...")
                continue
            img_height, img_width = img.shape[:2]

            # Copy the image
            shutil.copy(src_img_path, dst_img_path)

            # Generate label file
            label_path = os.path.join(output_labels, image_name.replace(".jpg", ".txt"))
            with open(label_path, 'w') as f:
                for label in entry.get("labels", []):
                    category = label.get("category")
                    if category not in YOLO_CLASSES:
                        continue

                    bbox = label.get("box2d")
                    if not bbox:
                        continue

                    x1 = bbox.get("x1")
                    y1 = bbox.get("y1")
                    x2 = bbox.get("x2")
                    y2 = bbox.get("y2")
                    if None in [x1, y1, x2, y2]:
                        continue

                    # Convert to YOLO format using actual image dimensions
                    yolo_box = convert_bbox_to_yolo([x1, y1, x2, y2], img_width, img_height)
                    class_id = YOLO_CLASSES[category]
                    f.write(f"{class_id} {' '.join(f'{x:.6f}' for x in yolo_box)}\n")

        print(f"✔ Processed {split} split: {len(selected)} images with YOLO annotations.")

# Run the filtering
filter_adverse_weather(BDD100K_ROOT, OUTPUT_DIR, MAX_IMAGES_TRAIN, MAX_IMAGES_VAL)

✔ Processed train split: 1000 images with YOLO annotations.
✔ Processed val split: 200 images with YOLO annotations.


## Step 2: Configure YOLOv5

Create a `data.yaml` file to specify the dataset paths and classes for YOLOv5 training.

In [3]:
# Create data.yaml file with augmentations
data_yaml_content = """
train: C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/dataset/images/train
val: C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/dataset/images/val
nc: 1
names: ['car']
# Augmentation parameters
hsv_h: 0.015  # Hue augmentation (fraction)
hsv_s: 0.7    # Saturation augmentation (fraction, increased to simulate weather effects)
hsv_v: 0.4    # Value (brightness) augmentation (fraction, increased for contrast)
degrees: 0.0  # Image rotation (+/- degrees)
translate: 0.1  # Image translation (+/- fraction)
scale: 0.5    # Image scale (+/- gain)
shear: 0.0    # Image shear (+/- degrees)
perspective: 0.0  # Image perspective (+/- fraction)
flipud: 0.0   # Image flip up-down (probability)
fliplr: 0.5   # Image flip left-right (probability)
mosaic: 1.0   # Mosaic augmentation (probability)
mixup: 0.0    # Mixup augmentation (probability)
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml_content)

print("data.yaml created successfully.")

data.yaml created successfully.


In [4]:
import os

# Check if yolov5 directory exists, if not clone it, otherwise pull the latest changes
if not os.path.exists("yolov5"):
    !git clone https://github.com/ultralytics/yolov5
    os.chdir("yolov5")
    !git checkout master
    !git pull
    os.chdir("..")
else:
    os.chdir("yolov5")
    !git checkout master
    !git pull
    os.chdir("..")

print("YOLOv5 updated to the latest version.")

Your branch is up to date with 'origin/master'.


Already on 'master'


Already up to date.
YOLOv5 updated to the latest version.


## Step 3: Train YOLOv5s

Train YOLOv5s on the filtered dataset for 10 epochs. Save the trained weights for later use.

In [5]:
import torch
import subprocess
import os

# Set the working directory to the project root
os.chdir("C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles")

# Define the training command as a list of arguments with absolute paths
train_command = [
    "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/.ML/Scripts/python.exe",
    "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/yolov5/train.py",
    "--data", "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/data.yaml",
    "--imgsz", "416",
    "--batch-size", "2",  # Reduced to avoid VRAM issues
    "--epochs", "10",
    "--weights", "yolov5s.pt",
    "--device", "0",  # Force GPU device 0
    "--project", "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs",
    "--name", "adverse_weather_exp",
    "--workers", "4"
]

# Propagate the current environment and add CUDA-related variables
env = os.environ.copy()
env["CUDA_VISIBLE_DEVICES"] = "0"  # Explicitly set visible GPU to device 0
result = subprocess.run(train_command, capture_output=True, text=True, env=env)
print("Training Output:")
print(result.stdout)
if result.stderr:
    print("Training Errors:")
    print(result.stderr)

# Check if training was successful
if result.returncode == 0:
    print("Training complete. Model weights saved to runs/adverse_weather_exp/weights/best.pt")
else:
    print("Training failed. See errors above.")

Exception in thread Thread-14 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\ATbru\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\ATbru\Downloads\CODE\AI_ML_DL\.ML\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\ATbru\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ATbru\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\ATbru\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 5856: char

Training Output:

Training complete. Model weights saved to runs/adverse_weather_exp/weights/best.pt


In [6]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

Torch version: 2.3.0+cu118
CUDA available: True
CUDA version: 11.8
GPU name: NVIDIA GeForce RTX 3050 Laptop GPU


## Step 4: Evaluate the Model

Evaluate the trained model to get mAP and save the results for later use.

In [7]:
import subprocess
import json
import os
import glob

# Define the validation command as a list of arguments
val_command = [
    "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/.ML/Scripts/python.exe",  # Corrected path to Python executable
    "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/yolov5/val.py",  # Absolute path to val.py
    "--weights", "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs/adverse_weather_exp/weights/best.pt",
    "--data", "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/data.yaml",
    "--imgsz", "416",
    "--device", "0" if torch.cuda.is_available() else "cpu",
    "--task", "val"
]

# Run the command and capture output
result = subprocess.run(val_command, capture_output=True, text=True)
print("Validation Output:")
print(result.stdout)
if result.stderr:
    print("Validation Errors:")
    print(result.stderr)

# Parse the last run's results (YOLOv5 saves them in runs/val/exp*/results.json)
val_exp_dir = max([d for d in glob.glob("C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs/val/exp*")], key=os.path.getmtime, default=None)
if val_exp_dir:
    results_path = os.path.join(val_exp_dir, "results.json")
    if os.path.exists(results_path):
        with open(results_path, "r") as f:
            metrics = json.load(f)
        # Save metrics to your custom file
        with open("C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/evaluation_metrics.json", "w") as f:
            json.dump(metrics, f)
        print("Evaluation complete. Metrics saved to evaluation_metrics.json")
        # YOLOv5 results.json typically contains mAP metrics
        print("mAP@0.5:", metrics.get("metrics/mAP50", "Not found"))
    else:
        print("Evaluation completed, but results.json not found in", val_exp_dir)
else:
    print("Evaluation failed. No validation run found. See errors above.")

Validation Output:

Validation Errors:
val: data=C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/data.yaml, weights=['C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs/adverse_weather_exp/weights/best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5\runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  v7.0-416-gfe1d4d99 Python-3.12.4 torch-2.3.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Traceback (most recent call last):
  File "C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\yolov5\val.py", line 604, in <module>
    main(opt)
  File "C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\yolov5\val.py", line 575, in main
    run(**vars(opt))
  File "C:\Users\ATbru\Downloads\CODE\AI_ML_DL\.ML\Lib\site-packages\torch\utils\_contextl

## Step 5: Run Inference and Save Results

Run inference on the training set and save the output images for presentation.

In [10]:
from yolov5 import detect

# Run inference
detect.run(
    weights='C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs/adverse_weather_exp5/weights/best.pt',
    source='C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/dataset/images/train',
    imgsz=(416, 416),  # Changed to tuple
    conf_thres=0.4,
    device=0 if torch.cuda.is_available() else 'cpu',
    project='C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs',
    name='infer',
    save_txt=True,
    save_conf=True
)

print("Inference complete. Results saved to runs/infer")

YOLOv5  v7.0-416-gfe1d4d99 Python-3.12.4 torch-2.3.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/6285 C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\dataset\images\train\00091078-84635cf2.jpg: 256x416 5 cars, 393.0ms
image 2/6285 C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\dataset\images\train\00091078-c1d32eea.jpg: 256x416 7 cars, 31.0ms
image 3/6285 C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\dataset\images\train\00091078-f32de4d2.jpg: 256x416 4 cars, 31.0ms
image 4/6285 C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\dataset\images\train\0010bf16-9ee17cd9.jpg: 256x416 3 cars, 32.0ms
image 5/6285 C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\dataset\images\train\0024b742-acbed4fb.jpg: 256x416 5 cars, 31.0ms
image 6/6285 C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\dataset\images\train\002f8552-0cdd5

Inference complete. Results saved to runs/infer


## Step 6: Display Results (For Presentation)

Load and display the saved results (metrics and inference outputs) without retraining.

In [15]:
from IPython.display import Image, display
import pandas as pd

# Load the results.csv file
results_path = "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs/adverse_weather_exp5/results.csv"
try:
    metrics_df = pd.read_csv(results_path, skipinitialspace=True, delimiter=',')
except FileNotFoundError:
    print(f"File not found: {results_path}")
    raise
except Exception as e:
    print(f"Error loading CSV: {e}")
    raise

# Strip whitespace from column names
metrics_df.columns = metrics_df.columns.str.strip()

# Print debug information
print("DataFrame Info:")
print(metrics_df.info())
print("\nDataFrame Head:")
print(metrics_df.head())
print("\nAvailable Columns:")
print(metrics_df.columns.tolist())

# Get the last row (final metrics)
final_metrics = metrics_df.iloc[-1]

# Extract key metrics
metrics = {
    "precision": final_metrics["metrics/precision"],
    "recall": final_metrics["metrics/recall"],
    "mAP@0.5": final_metrics["metrics/mAP_0.5"],
    "mAP@0.5:0.95": final_metrics["metrics/mAP_0.5:0.95"]
}

print("\nLoaded Evaluation Metrics:")
print(metrics)

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   epoch                 10 non-null     int64  
 1   train/box_loss        10 non-null     float64
 2   train/obj_loss        10 non-null     float64
 3   train/cls_loss        10 non-null     int64  
 4   metrics/precision     10 non-null     float64
 5   metrics/recall        10 non-null     float64
 6   metrics/mAP_0.5       10 non-null     float64
 7   metrics/mAP_0.5:0.95  10 non-null     float64
 8   val/box_loss          10 non-null     float64
 9   val/obj_loss          10 non-null     float64
 10  val/cls_loss          10 non-null     int64  
 11  x/lr0                 10 non-null     float64
 12  x/lr1                 10 non-null     float64
 13  x/lr2                 10 non-null     float64
dtypes: float64(11), int64(3)
memory usage: 1.2 KB
None

DataFrame

## Step 7: Test on New Image (For Presentation)

Load the saved model and test it on a new image (e.g., one you take). Update the `new_image_path` to the new image file before running this cell.

In [17]:
import os
import torch
from yolov5 import detect
from IPython.display import Image, display

# Paths
weights_path = "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs/adverse_weather_exp5/weights/best.pt"
new_image_path = "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/rr.jpeg"  # Replace with the actual path to your new image
new_infer_dir = "C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs/new_infer"

# Create directory for new inference results
os.makedirs(new_infer_dir, exist_ok=True)

# Run inference on the new image
detect.run(
    weights=weights_path,
    source=new_image_path,
    imgsz=(416, 416),  # Use tuple for image size
    conf_thres=0.4,
    device=0 if torch.cuda.is_available() else 'cpu',
    project='C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs',
    name='new_infer',
    save_txt=True,
    save_conf=True,
    nosave=False  # Ensure images are saved (optional, defaults to False)
)

print("Inference on new image complete. Results saved to runs/new_infer")

# Display the output image
output_image_path = os.path.join(new_infer_dir, os.path.basename(new_image_path))  # Matches the input image name
if os.path.exists(output_image_path):
    display(Image(filename=output_image_path))
else:
    print(f"Output image not found at {output_image_path}. Check the runs/new_infer directory for saved files.")

YOLOv5  v7.0-416-gfe1d4d99 Python-3.12.4 torch-2.3.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\rr.jpeg: 416x192 (no detections), 115.0ms
Speed: 0.0ms pre-process, 115.0ms inference, 0.0ms NMS per image at shape (1, 3, 416, 416)
Results saved to C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\runs\new_infer2
0 labels saved to C:\Users\ATbru\Downloads\CODE\AI_ML_DL\Autonomus_Vehicles\runs\new_infer2\labels


Inference on new image complete. Results saved to runs/new_infer
Output image not found at C:/Users/ATbru/Downloads/CODE/AI_ML_DL/Autonomus_Vehicles/runs/new_infer\rr.jpeg. Check the runs/new_infer directory for saved files.


## Conclusion

- Successfully trained YOLOv5s on ~1,000 BDD100K images with adverse weather conditions.
- Achieved an mAP of approximately 0.2-0.3 (due to dummy labels).
- Saved model weights, metrics, and inference outputs for presentation.
- Ready to show results to the professor without retraining.